In [9]:
import cv2

TEXT_PATH = './data/data.txt'

class Video_capture:
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.ret = None 
        self.frame = None 
        
    def move_frame(self, frame_cnt):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_cnt)
        self.read()
        
    def make_grayscale(self):
        return cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
    
    def read(self):
        self.ret, self.frame = self.cap.read()
        return

class Highlight_finder:
    def __init__(self, source, highlight):
        self.name = source
        self.source = Video_capture(source)
        self.highlight= Video_capture(highlight)
        self.frame_cnt = 0
        self.highlight_frame_cnt = 210 # 네이버 로고 지난 프레임
        self.result = []
    
    def load_video(self):
        if not self.source.cap.isOpened():
                print('source video not opened')
                exit(-1)
        if not self.highlight.cap.isOpened():
                print('highlight video not opened')
                exit(-1)
        self.source.move_frame(self.frame_cnt)
        self.highlight.move_frame(self.highlight_frame_cnt)       
        return
    
    def match_template(self, src, frame):
        R = cv2.matchTemplate(src, frame, cv2.TM_CCORR_NORMED)
        _, maxVal, _, maxLoc = cv2.minMaxLoc(R)
        return maxVal
    
    def find_highlight(self):
        start = 0
        end = 0
        f = open(self.name +".txt",'w')
        while True: # 영상이 끝날때까지
            self.frame_cnt += 5
            if self.frame_cnt % 1800 == 0:
                print(self.frame_cnt / 1800, "min", self.highlight_frame_cnt)
            for i in range(5):
                self.source.read()
            if not self.source.ret or not self.highlight.ret:
                break
        #---------------------------------------------------------------------
            maxVal = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())

            #cv2.imshow('source', self.source.frame)
            key = cv2.waitKey(25)
            if key == 27:
                cv2.destroyAllWindows()
                break

            if maxVal < 0.95: # 영상이 일치하는지 체크
                continue
            else: # 일치한다면
                result = maxVal
                if(self.frame_cnt - end > 150):
                    if end != 0 and end - start >= 60:
                        start_time = str(start / 30 / 3600) + ":" + str(start/30%3600/60) + ":"str(start/30%60)
                        end_time = str(end / 30 / 3600) + ":" + str(end/30%3600/60) + ":"str(end/30%60)
                        f.write(str(start) + "," + str(end)+"," + start_time + "," + end_time+"\n")
                    start = self.frame_cnt
                while True: # 일치하지 않는 구간까지 반복
                    print(self.frame_cnt, result, self.highlight_frame_cnt)
                    self.highlight_frame_cnt += 15
                    self.frame_cnt += 15
                    for i in range(15):
                        self.highlight.read()
                        self.source.read()
                        
                    '''
                    cv2.imshow('source', self.source.frame)
                    cv2.imshow('highlight', self.highlight.frame)
                    key = cv2.waitKey(25)
                    if key == 27:
                        cv2.destroyAllWindows()
                    '''
                    if not self.source.ret or not self.highlight.ret: 
                        break
                    result = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())
                    if result > 0.95: # 일치한다면
                        continue
                    else: # 일치하지 않는다면
                        print(self.frame_cnt, result, self.highlight_frame_cnt)
                        end = self.frame_cnt - 15
                        self.highlight_frame_cnt += 30
                        for i in range(30):
                            self.highlight.read()
                        break
                
        #----------------------------------------------------------------------
        if self.source.cap.isOpened():
            self.source.cap.release()

        if self.highlight.cap.isOpened():
            self.highlight.cap.release()
            
        f.write(str(start) + "," + str(end))
        f.close()
        #cv2.destroyAllWindows()
        print("END")
        return

In [10]:
with open("./data/data.txt", "r") as src:
    while(True):
        source_path = src.readline()
        if not source_path : break
        source_path = "./data/" + source_path.split(',')[1].strip()
        highlight_path = source_path + "_HL.mp4"
        source_path += ".mp4"
        print(source_path)
        finder = Highlight_finder(source_path, highlight_path)
        finder.load_video()
        finder.find_highlight()

./data/20200411_APK_HLE_1.mp4
1.0 min 210
2.0 min 210
3.0 min 210
4.0 min 210
5.0 min 210
6.0 min 210
7.0 min 210
8.0 min 210
9.0 min 210
10.0 min 210
11.0 min 210
12.0 min 210
13.0 min 210
14.0 min 210
26100 0.9992541074752808 210
26115 0.9995412230491638 225
26130 0.6095271110534668 240
15.0 min 270
27135 0.9998067021369934 270
27150 0.9998586177825928 285
27165 0.9998430609703064 300
27180 0.9858434796333313 315
27195 0.8087942004203796 330
27220 0.950735330581665 360
27235 0.999923586845398 375
27250 0.9999533891677856 390
27265 0.9999532103538513 405
27280 0.7956535816192627 420
27315 0.9626916646957397 450
27330 0.9766311049461365 465
27345 0.9673756957054138 480
27360 0.8862839937210083 495
16.0 min 525
29920 0.9530746936798096 525
29935 0.9590139389038086 540
29950 0.9727398753166199 555
29965 0.9844117760658264 570
29980 0.9502577185630798 585
29995 0.9476348161697388 600
30025 0.9519693851470947 630
30040 0.9594108462333679 645
30055 0.956134021282196 660
30070 0.957538604736

57920 0.9858401417732239 4545
57935 0.9628856778144836 4560
57950 0.9624344706535339 4575
57965 0.9678134322166443 4590
57980 0.8128702640533447 4605
58605 0.9533566236495972 4635
58620 0.9628902077674866 4650
58635 0.9724001288414001 4665
58650 0.9715849161148071 4680
58665 0.968651294708252 4695
58680 0.9679559469223022 4710
58695 0.9556432366371155 4725
58710 0.9433366060256958 4740
58740 0.9728255271911621 4770
58755 0.986173689365387 4785
58770 0.985918402671814 4800
58785 0.9860721826553345 4815
58800 0.9810507893562317 4830
58815 0.9778123497962952 4845
58830 0.968334972858429 4860
58845 0.952500581741333 4875
58860 0.9716712236404419 4890
58875 0.9017395377159119 4905
58885 0.9554226398468018 4935
58900 0.8988111615180969 4950
33.0 min 4980
34.0 min 4980
35.0 min 4980
64475 0.9511910676956177 4980
64490 0.9371249079704285 4995
64570 0.9533920288085938 5025
64585 0.9558532238006592 5040
64600 0.9568238258361816 5055
64615 0.951143741607666 5070
64630 0.9498233199119568 5085
6464

83405 0.9418147206306458 9150
83460 0.9778767824172974 9180
83475 0.9866806268692017 9195
83490 0.9787617921829224 9210
83505 0.9850945472717285 9225
83520 0.9803096055984497 9240
83535 0.9836599230766296 9255
83550 0.9855131506919861 9270
83565 0.9791304469108582 9285
83580 0.9815123677253723 9300
83595 0.990129828453064 9315
83610 0.9873839020729065 9330
83625 0.9728403687477112 9345
83640 0.9856582880020142 9360
83655 0.9811527132987976 9375
83670 0.9758265018463135 9390
83685 0.9806215763092041 9405
83700 0.9869917631149292 9420
83715 0.9896210432052612 9435
83730 0.9846183061599731 9450
83745 0.988986611366272 9465
83760 0.9870598316192627 9480
83775 0.9907848238945007 9495
83790 0.9888813495635986 9510
83805 0.984587550163269 9525
83820 0.983042299747467 9540
83835 0.990695059299469 9555
83850 0.9931237697601318 9570
83865 0.9910244345664978 9585
83880 0.9912869930267334 9600
83895 0.9878315329551697 9615
83910 0.9870103001594543 9630
83925 0.9898858070373535 9645
83940 0.9834754

12530 0.9989389777183533 270
12545 0.9998605847358704 285
12560 0.9998975992202759 300
12575 0.9998981952667236 315
12590 0.9998982548713684 330
12605 0.7648941278457642 345
12625 0.974138617515564 375
12640 0.979981541633606 390
12655 0.977974534034729 405
12670 0.9606778025627136 420
12685 0.8039304614067078 435
8.0 min 465
15085 0.9568344950675964 465
15100 0.9505972862243652 480
15115 0.9401353597640991 495
15195 0.9501038789749146 525
15210 0.9477861523628235 540
15245 0.9546259641647339 570
15260 0.9798544645309448 585
15275 0.9821498394012451 600
15290 0.9744713306427002 615
15305 0.9645355343818665 630
15320 0.973861575126648 645
15335 0.9563957452774048 660
15350 0.9450923204421997 675
15355 0.951575756072998 705
15370 0.9444754719734192 720
15445 0.9626866579055786 750
15460 0.9666470289230347 765
15475 0.9659423232078552 780
15490 0.9695279002189636 795
15505 0.9777972102165222 810
15520 0.9764038324356079 825
15535 0.9780628681182861 840
15550 0.9325064420700073 855
15560 0

23.0 min 4860
24.0 min 4860
25.0 min 4860
26.0 min 4860
27.0 min 4860
28.0 min 4860
29.0 min 4860
52760 0.950880765914917 4860
52775 0.9507522583007812 4875
52790 0.9451050162315369 4890
52855 0.9510082602500916 4920
52870 0.9556800723075867 4935
52885 0.9629002213478088 4950
52900 0.9655761122703552 4965
52915 0.9653199911117554 4980
52930 0.9770307540893555 4995
52945 0.9525458812713623 5010
52960 0.9595476984977722 5025
52975 0.9343591332435608 5040
53025 0.9525085687637329 5070
53040 0.9563634991645813 5085
53055 0.958318829536438 5100
53070 0.9569783210754395 5115
53085 0.9506394863128662 5130
53100 0.9493408203125 5145
53140 0.9545829892158508 5175
53155 0.9538687467575073 5190
53170 0.9477700591087341 5205
53205 0.95283442735672 5235
53220 0.9582890272140503 5250
53235 0.9549696445465088 5265
53250 0.9609617590904236 5280
53265 0.9661058187484741 5295
53280 0.9649784564971924 5310
53295 0.9646097421646118 5325
53310 0.9283472299575806 5340
30.0 min 5370
31.0 min 5370
57505 0.950

73975 0.9915543794631958 9435
73990 0.9721108078956604 9450
74005 0.984197199344635 9465
74020 0.9723559021949768 9480
74035 0.9700896143913269 9495
74050 0.9664322733879089 9510
74065 0.9731056690216064 9525
74080 0.9744376540184021 9540
74095 0.9864178895950317 9555
74110 0.9776182174682617 9570
74125 0.9815095663070679 9585
74140 0.9808530807495117 9600
74155 0.98162841796875 9615
74170 0.9830281138420105 9630
74185 0.9877462983131409 9645
74200 0.9867181777954102 9660
74215 0.9922553300857544 9675
74230 0.9874459505081177 9690
74245 0.9916026592254639 9705
74260 0.9701195359230042 9720
74275 0.992356538772583 9735
74290 0.990330696105957 9750
74305 0.9888491630554199 9765
74320 0.9874801635742188 9780
74335 0.9897411465644836 9795
74350 0.9877036213874817 9810
74365 0.9897516369819641 9825
74380 0.9832085967063904 9840
74395 0.9882725477218628 9855
74410 0.9194590449333191 9870
74430 0.9516822695732117 9900
74445 0.9015251994132996 9915
42.0 min 9945
76930 0.9554774165153503 9945
7

85340 0.9977349042892456 13605
85355 0.9981831312179565 13620
85370 0.9976170659065247 13635
85385 0.9981703758239746 13650
85400 0.9988841414451599 13665
85415 0.9986317157745361 13680
85430 0.9987829327583313 13695
85445 0.9991090893745422 13710
85460 0.9992550015449524 13725
85475 0.9990813732147217 13740
85490 0.8223746418952942 13755
48.0 min 13785
49.0 min 13785
50.0 min 13785
51.0 min 13785
92850 0.9543331861495972 13785
92865 0.9937431812286377 13800
92880 0.9994803071022034 13815
92895 0.9994845390319824 13830
92910 0.9994850754737854 13845
92925 0.9995034337043762 13860
92940 0.9995377063751221 13875
92955 0.9995435476303101 13890
92970 0.9995430707931519 13905
92985 0.9995227456092834 13920
93000 0.999495267868042 13935
93015 0.9995022416114807 13950
93030 0.9995024800300598 13965
93045 0.9995030164718628 13980
93060 0.9995515942573547 13995
93075 0.9995551705360413 14010
93090 0.9995554685592651 14025
93105 0.9995051026344299 14040
93120 0.9994801878929138 14055
93135 0.753

39455 0.960785448551178 4185
39470 0.9582804441452026 4200
39485 0.9587975740432739 4215
39500 0.950395941734314 4230
39515 0.9498107433319092 4245
39555 0.9504884481430054 4275
39570 0.9479904174804688 4290
39585 0.9517126679420471 4320
39600 0.9566957354545593 4335
39615 0.9448350071907043 4350
39650 0.9533947706222534 4380
39665 0.906666100025177 4395
39710 0.952067494392395 4425
39725 0.9660221934318542 4440
39740 0.9702308177947998 4455
39755 0.9699639678001404 4470
39770 0.9593948721885681 4485
39785 0.965155303478241 4500
39800 0.9610605239868164 4515
39815 0.9342493414878845 4530
39835 0.9622185230255127 4560
39850 0.9513453841209412 4575
39865 0.9130697846412659 4590
40895 0.951582133769989 4620
40910 0.9492442011833191 4635
40915 0.9663123488426208 4665
40930 0.9596810936927795 4680
40945 0.9498049020767212 4695
40975 0.9512795805931091 4725
40990 0.9503461122512817 4740
41005 0.9486207962036133 4755
41010 0.9541469216346741 4785
41025 0.9374815821647644 4800
41165 0.98760092

63285 0.9916489720344543 8790
63300 0.994297981262207 8805
63315 0.9878902435302734 8820
63330 0.8288949131965637 8835
63815 0.9577011466026306 8865
63830 0.9417740106582642 8880
63860 0.9541519284248352 8910
63875 0.9571062922477722 8925
63890 0.9604018926620483 8940
63905 0.9557899832725525 8955
63920 0.9263755083084106 8970
63950 0.9539351463317871 9000
63965 0.960461437702179 9015
63980 0.9386920928955078 9030
64015 0.9565086364746094 9060
64030 0.9744179844856262 9075
64045 0.9629942178726196 9090
64060 0.945500373840332 9105
64095 0.9614804983139038 9135
64110 0.9710513949394226 9150
64125 0.9568791389465332 9165
64140 0.9673231244087219 9180
64155 0.9759906530380249 9195
64170 0.9733532071113586 9210
64185 0.9734808206558228 9225
64200 0.9646975994110107 9240
64215 0.972143828868866 9255
64230 0.9729245901107788 9270
64245 0.9820328950881958 9285
64260 0.9773842096328735 9300
64275 0.9655944108963013 9315
64290 0.986459493637085 9330
64305 0.9805408120155334 9345
64320 0.9854506

81185 0.9639250636100769 13245
81200 0.9563683271408081 13260
81215 0.9644455313682556 13275
81230 0.969596266746521 13290
81245 0.9885368943214417 13305
81260 0.9714595079421997 13320
81275 0.9719891548156738 13335
81290 0.9871810078620911 13350
81305 0.9924243092536926 13365
81320 0.9943258166313171 13380
81335 0.8442502617835999 13395
81340 0.9554955959320068 13425
81355 0.5655507445335388 13440
81400 0.9501996636390686 13470
81415 0.9916027784347534 13485
81430 0.9879167675971985 13500
81445 0.984190821647644 13515
81460 0.9764605760574341 13530
81475 0.9894477725028992 13545
81490 0.9734012484550476 13560
81505 0.9846140146255493 13575
81520 0.9828261733055115 13590
81535 0.9905291199684143 13605
81550 0.9837961196899414 13620
81565 0.9866656064987183 13635
81580 0.9866441488265991 13650
81595 0.9848014712333679 13665
81610 0.9824591279029846 13680
81625 0.9857838153839111 13695
81640 0.9886444211006165 13710
81655 0.9905549883842468 13725
81670 0.988309919834137 13740
81685 0.979